# Test Model

### Notes:
1. Output metrics need to be given in SDR too
2. SNR's below 0 have high noise levels instead of low music levels - signal levels should be more randomised throughout the whole traning set. I'll to this by normalising the waveform after adding the noise, and then randomising the level to between 0 and -5 dB - this will allow me to double the training data from MUSDB18 by augmenting with different levels and noise masking.
3. use just MUSDB18 so I can present this as research.   
4. Given that I'm using the ESC50 set, i should be able to find what noise is what so I can label them when evaluating

In [1]:
import sys
import os

# Detect Google Colab
if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Detect Google Colab
if "google.colab" in sys.modules:
    print("Running in Google Colab...")
    os.system("git clone https://github.com/CiaranMaloy/audioautoencoder")
    os.chdir("/content/audioautoencoder/")
    os.system("git pull")
    os.system("git checkout dataset-generation-fix")
    os.system("git pull origin dataset-generation-fix")
    #os.system("pip install --upgrade torchmetrics")
else:
    print("Running locally...")
    os.system("git pull origin dataset-generation-fix")
    #os.system("pip install --upgrade torchmetrics")


Running in Google Colab...


In [3]:
import sys
sys.path.append('/content/audioautoencoder')
sys.path.append('/content/audioautoencoder/audioautoencoder')

Notes on testing:
1. SDR is missing as a metric

## Load Model

In [4]:
from audioautoencoder.models.UNetConv4 import *
from audioautoencoder.training import DenoisingLoader

In [5]:
import torch

In [6]:
model_name = 'UNetConv4'
SNRdB_load = [-10, 10]
SNRdB = SNRdB_load
load_file = 'Autoencodermodel_earlystopping.pth'


load_path = f'/content/drive/MyDrive/Projects/ML_Projects/De-noising-autoencoder/Models_Comparison/Checkpoints_{model_name}_{SNRdB_load[0]}-{SNRdB_load[1]}/{load_file}'

In [7]:
# Add the custom class to the safe globals list
torch.serialization.add_safe_globals([UNetConv4])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNetConv4(in_channels=4, out_channels=4).to(device)

# Now load your checkpoint normally
denoiser = DenoisingLoader(model, load_path)
model = denoiser.model
print('Loaded Model')

# Example input (batch_size=1, channels=2, height=1025, width=175)
noisy_input = torch.randn(2, 4, 1025 // 4, 175)

denoised_output = denoiser.denoise(noisy_input)
print(denoised_output.shape)

Loaded model from /content/drive/MyDrive/Projects/ML_Projects/De-noising-autoencoder/Models_Comparison/Checkpoints_UNetConv4_-10-10/Autoencodermodel_earlystopping.pth
Loaded Model
torch.Size([2, 4, 256, 175])


In [8]:
IMPORT_TEST_NOISY = True
load_dataframe = False
max_file_size_gb = 6

In [9]:
from audioautoencoder.datasets.utils import *
from audioautoencoder.data import *
from audioautoencoder.data_management import *
from audioautoencoder.generate_dataset import *

In [10]:
train_diffusion = False
SNRdB_load = [-10, 10]
SNRdBs = [[-10, 10]] # SNR random range
#load_file = 'Autoencodermodel_earlystopping.pth'
load_file = 'Autoencodermodel_checkpoint.pth'

folder = ['sep_features', 'all-noise_features', 'all-noise_features_2'][2] # sep

output_path = f'/content/drive/MyDrive/Projects/ML_Projects/De-noising-autoencoder/Models_Comparison/Checkpoints_{model_name}_{SNRdB_load[0]}-{SNRdB_load[1]}/'
scaler_file = output_path + "scalers.pkl"  # Static filename since it's unique per run
os.makedirs(os.path.dirname(scaler_file), exist_ok=True)
source_folder = f"/content/drive/MyDrive/Datasets/Music-Noise/SNRdB_{folder}/SNRdB_{SNRdB[0]}-{SNRdB[1]}/"
source_path = source_folder + "test/"

In [11]:
destination_path = f"/content/SNRdB_{SNRdB[0]}-{SNRdB[1]}/test/"

from google.colab import drive

if IMPORT_TEST_NOISY:
  if not os.path.exists(destination_path):
    combine_h5_files_spectrograms(source_path, destination_path, max_file_size_gb=max_file_size_gb)

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Datasets/Music-Noise/SNRdB_all-noise_features_2/SNRdB_-10-10/test/'

In [ ]:
from audioautoencoder.datasets.utils import *

In [ ]:
dataset_path = f"/content/SNRdB_{SNRdB[0]}-{SNRdB[1]}/test/combined_000.h5"

In [ ]:
if os.path.exists(scaler_file):
    print("Loading existing scalers...")
    scalers = load_scalers(scaler_file)
else:
    print("Training new scalers...")
    scalers = train_scalers_separation(dataset_path, sample_size=8000)
    save_scalers(scalers, scaler_file)

In [ ]:
if IMPORT_TEST_NOISY:
    print("Loading existing scalers...")
    scalers = load_scalers(scaler_file)
    test_loader = ChannelDatasetLoader(
          dataset_path=dataset_path,
          scalers=scalers,
          output_time_length=175,
          channels=1,
          snr_db=SNRdB,
          subset=False,
          batch_size=4
      )

    print(f"Training set size: {len(test_loader.train_dataset)}")
    print(f"Validation set size: {len(test_loader.val_dataset)}")

In [ ]:
import os
os.system("pip install --upgrade torchmetrics")

In [ ]:
from audioautoencoder.testing import *

In [ ]:
if load_dataframe:
  df_subset = pd.read_csv(output_path + f"df_subset_SNRdB_{SNRdB[0]}-{SNRdB[1]}.csv")

## Test Model

Use a large amount of data for a general L1 score to be recorded as an overall performance

In [ ]:
if not load_dataframe:
  criterion = nn.L1Loss()
  loss = test_model_gpu(model, test_loader.train_loader, criterion, scalers)
  print(f"Test Loss: {loss}")

Use the val split as data for the more processing intensive df_eval generation, for a more in depth performance analysis

In [ ]:
if not load_dataframe:
  criterion = nn.L1Loss()
  loss, df_eval = test_model(model, test_loader.val_loader, criterion, scalers)

In [ ]:
if not load_dataframe:
  # Assuming `df` is your original dataframe
  #df_eval["Improvement"] = df_eval["l1_outvstar"] df_eval["l1_invstar"]  # Higher SDR is better
  subset_columns = ["instance", "l1_invstar", "l1_outvstar", "l1_invstar_4k", "l1_outvstar_4k", "l1_invstar_full", "l1_outvstar_full",  "filename", "snr_db"] #"Improvement"]
  df_subset = df_eval#[subset_columns]

In [ ]:
if not load_dataframe:
  # Create a function to map filename to a class
  def get_class_from_filename(filename, classes):
      for keyword in classes:
          if keyword in filename:
              return keyword
      return 'Unknown'  # Default if no match found

  df_subset[['filename_audio', 'filename_noise']] = pd.DataFrame(df_subset['filename'].tolist(), index=df_subset.index)
  df_subset['filename_audio'] = df_subset['filename_audio'].apply(lambda x: x.decode('utf-8'))
  df_subset['filename_noise'] = df_subset['filename_noise'].apply(lambda x: x.decode('utf-8'))

  classes = ['mixture', 'vocals', 'drums', 'guitar', 'bass', 'piano', 'electric_guitar', 'acoustic_guitar', 'synthesizer', 'strings', 'brass']
  df_subset['audio_class'] = df_subset['filename_audio'].apply(lambda x: get_class_from_filename(x, classes))

  classes = ['0707', 'Rain', 'Crowd', 'Water', 'Ice']
  df_subset['noise_class'] = df_subset['filename_noise'].apply(lambda x: get_class_from_filename(x, classes))


In [ ]:
df_subset["Improvement_L1"] = df_subset["l1_invstar"] - df_subset["l1_outvstar"]  # Lower L1 loss is better
df_subset["Improvement_L1_4k"] = df_subset["l1_invstar_4k"] - df_subset["l1_outvstar_4k"]  # Lower L1 loss is better
df_subset["Improvement_L1_full"] = df_subset["l1_invstar_full"] - df_subset["l1_outvstar_full"]  # Lower L1 loss is better

## Plot Results

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


# Set minimal theme
sns.set_theme(style="white", font_scale=1.2)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df_eval = df_subset.copy()

# Round SNR to the nearest 2.5 dB
df_eval["snr_db_rounded"] = (df_eval["snr_db"] / 2.5).round() * 2.5

def compute_stats(df, class_col):
    grouped = df.groupby(["snr_db_rounded", class_col])["Improvement_L1"]
    stats = grouped.agg(["mean", "std", "count"]).reset_index()
    stats.rename(columns={
        "mean": "mean_improvement",
        "std": "std_improvement",
        "count": "count"
    }, inplace=True)
    return stats

df_audio_stats = compute_stats(df_eval, "audio_class")
df_noise_stats = compute_stats(df_eval, "noise_class")


In [ ]:
import numpy as np
plt.figure(figsize=(10, 6))

# Calculate weighted mean and std using counts
df_noise_stats["weighted_var"] = df_noise_stats["std_improvement"]**2 * df_noise_stats["count"]

grouped_all = df_noise_stats.groupby("snr_db_rounded").agg(
    mean_improvement=("mean_improvement", lambda x: np.average(x, weights=df_noise_stats.loc[x.index, "count"])),
    std_improvement=("weighted_var", lambda x: np.sqrt(np.sum(x) / np.sum(df_noise_stats.loc[x.index, "count"]))),
    total_count=("count", "sum")
).reset_index()

# Filter "Crowd" class
crowd_df = df_noise_stats[df_noise_stats["noise_class"] == "Crowd"]

# Plot weighted average
plt.plot(
    grouped_all["snr_db_rounded"],
    grouped_all["mean_improvement"],
    marker="o",
    label="Weighted Mean (All Classes)",
    color="steelblue"
)
plt.errorbar(
    grouped_all["snr_db_rounded"],
    grouped_all["mean_improvement"],
    yerr=grouped_all["std_improvement"],
    fmt='none',
    capsize=3,
    color="steelblue",
    alpha=0.7
)

offset = 0.25
# --- Plot "Crowd" noise class (with slight offset) ---
plt.plot(
    crowd_df["snr_db_rounded"] + offset,
    crowd_df["mean_improvement"],
    marker="s",
    label="Crowd Noise",
    color="darkorange"
)
plt.errorbar(
    crowd_df["snr_db_rounded"] + offset,
    crowd_df["mean_improvement"],
    yerr=crowd_df["std_improvement"],
    fmt='none',
    capsize=3,
    color="darkorange",
    alpha=0.7
)

plt.xlabel("SNR (dB, rounded to nearest 2.5)")
plt.ylabel("Mean Improvement (L1)")
plt.title("Mean Improvement vs SNR\n(Weighted Overall vs Crowd Noise)")
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()
plt.ylim((-0.2, 0.4))
plt.show()


In [ ]:
# Recalculate and clean labels
df_eval["snr_db_rounded"] = (df_eval["snr_db"] / 2.5).round() * 2.5

# Clean string labels and keep mapping for sorting
df_eval["snr_db_rounded_str"] = df_eval["snr_db_rounded"].apply(
    lambda x: f"{x:.1f}".rstrip('0').rstrip('.') if '.' in f"{x:.1f}" else f"{int(x)}"
)

# Sort labels numerically
ordered_snrs = sorted(df_eval["snr_db_rounded"].unique())
ordered_labels = [f"{x:.1f}".rstrip('0').rstrip('.') if '.' in f"{x:.1f}" else f"{int(x)}" for x in ordered_snrs]

plt.figure(figsize=(12, 6))
sns.boxplot(
    data=df_eval,
    x="snr_db_rounded_str",
    y="Improvement_L1",
    hue="noise_class",
    palette="tab10",
    order=ordered_labels  # ✅ apply numeric order
)

plt.xlabel("SNR (dB, rounded to nearest 2.5)")
plt.ylabel("Improvement (L1)")
plt.title("Distribution of Improvement vs SNR by Noise Class (Box Plot)")
plt.legend(title="Noise Class", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.ylim((-0.2, 0.4))
plt.show()





In [ ]:

plt.figure(figsize=(12, 6))
sns.boxplot(
    data=df_eval,
    x="snr_db_rounded_str",
    y="Improvement_L1",
    hue="noise_class",
    palette="tab10",
    order=ordered_labels,
    showfliers=False,  # Hide outliers
    linewidth=0.8,
    width=0.6
)

plt.xlabel("SNR (dB, rounded to nearest 2.5)")
plt.ylabel("Improvement (L1)")
plt.title("Distribution of Improvement vs SNR by Noise Class (Box Plot)")
plt.legend(title="Noise Class", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.ylim((-0.2, 0.4))
plt.show()


In [ ]:
g = sns.catplot(
    data=df_eval,
    x="snr_db_rounded_str",
    y="Improvement_L1",
    col="noise_class",
    kind="box",
    col_wrap=3,
    order=ordered_labels,
    color="skyblue",  # ✅ uniform color for all boxes
    height=4,
    aspect=1
)

# Titles, labels, limits
g.set_titles(col_template="{col_name}")
g.set_axis_labels("SNR (dB)", "Improvement (SDR)")
g.set(ylim=(-0.2, 0.6))

# ✅ Add horizontal gridlines to all facets
for ax in g.axes.flat:
    ax.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
# Save subset dataframe
df_subset.to_csv(output_path + f"df_subset_SNRdB_{SNRdB[0]}-{SNRdB[1]}.csv", index=False)

In [ ]:
import gc

# Delete large variables

# Force garbage collection
gc.collect()

In [ ]:
def threshold_spectrogram(spectrogram, threshold):
    """
    Zeroes out all values in the spectrogram that are below the given threshold.

    Args:
        spectrogram (np.ndarray): Input 2D array.
        threshold (float): The threshold value.

    Returns:
        np.ndarray: The processed spectrogram with values below threshold set to zero.
    """
    spectrogram = np.where(spectrogram >= threshold, spectrogram, 0)
    return spectrogram

## View Spectrogram

In [ ]:
sampling_rate = 44100  # 44.1 kHz audio
n_fft = 2048  # Adjust this for better resolution
freqs = np.linspace(0, sampling_rate / 2, n_fft // 2 + 1)  # STFT frequency bins

# Find indices corresponding to 0–4000 Hz
min_freq, max_freq = 0, 4000
freq_indices = np.where((freqs >= min_freq) & (freqs <= max_freq))[0]

# in spectrogram
index = 40

snr_db = np.array(df_subset.loc[index, "snr_db"])
print(snr_db)

# lets evaluate this from a l1 loss perspective
# reconstruct spectrogram
out_spectrogram = np.array(df_subset.loc[index, "out_track"][0])
out_spectrogram[df_subset.loc[index, "metadata"]["freq_indices_hf"], :] = resample_feature(np.array(df_subset.loc[index, "out_track"][1]), df_subset.loc[index, "metadata"]["hf_shape"])
out_spectrogram[df_subset.loc[index, "metadata"]["freq_indices_mf"], :] = resample_feature(np.array(df_subset.loc[index, "out_track"][2]), df_subset.loc[index, "metadata"]["mf_shape"])
out_spectrogram[df_subset.loc[index, "metadata"]["freq_indices_lf"], :] = resample_feature(np.array(df_subset.loc[index, "out_track"][3]), df_subset.loc[index, "metadata"]["lf_shape"])
out_spec_copy = out_spectrogram

out_spectrogram = threshold_spectrogram(out_spectrogram, np.mean(out_spectrogram)*0.75)

# out, with no join
out_track = np.array(df_subset.loc[index, "out_track"])[0]

# out spectrogram
in_spectrogram = df_subset.loc[index, "in_track"][0]

# target
tar_track = np.array(df_subset.loc[index, "tar_track"])[0]

# inverse normalisation to 0 - 1
out_spectrogram = (out_spectrogram - 0.5) * 2
out_track = (out_track - 0.5) * 2
in_spectrogram = (in_spectrogram - 0.5) * 2
tar_track = (tar_track - 0.5) * 2

# Inverse standardisation
input_temp = tar_track
in_spectrogram = scalers["input_features_spectrogram"].inverse_transform(in_spectrogram.reshape(1, -1)).reshape(input_temp.shape)

out_spectrogram = scalers["target_features_spectrogram"].inverse_transform(out_spectrogram.reshape(1, -1)).reshape(input_temp.shape)
out_track = scalers["target_features_spectrogram"].inverse_transform(out_track.reshape(1, -1)).reshape(input_temp.shape)

tar_track = scalers["target_features_spectrogram"].inverse_transform(tar_track.reshape(1, -1)).reshape(input_temp.shape)

# plot things
# Plot spectrograms
fig, axes = plt.subplots(4, 1, figsize=(15, 15))

axes[0].imshow(in_spectrogram, aspect="auto", cmap="magma", origin="lower")
axes[0].set_title("Noisy Input (Log Scale)")
axes[0].set_yscale("log")
axes[0].set_ylim((1, 1000))

axes[1].imshow(out_spectrogram, aspect="auto", cmap="magma", origin="lower")
axes[1].set_title("Denoised Output (Log Scale) - reconstructed")
axes[1].set_yscale("log")
axes[1].set_ylim((1, 1000))

axes[2].imshow(out_track, aspect="auto", cmap="magma", origin="lower")
axes[2].set_title("Denoised Output (Log Scale)")
axes[2].set_yscale("log")
axes[2].set_ylim((1, 1000))

axes[3].imshow(tar_track, aspect="auto", cmap="magma", origin="lower")
axes[3].set_title("Clean Target (Log Scale)")
axes[3].set_yscale("log")
axes[3].set_ylim((1, 1000))

plt.tight_layout()
plt.show()

In [ ]:
def magphase_to_waveform(magnitude, phase, audio_length=44100):
    """
    Converts a spectrogram image back into an audio waveform.

    Parameters:
        image (np.array): Spectrogram image (3 channels).
        sr (int): Sampling rate.

    Returns:
        np.array: Reconstructed audio waveform.
    """
    stft = magnitude * np.exp(1j * phase)
    return librosa.istft(stft, length=audio_length)

In [ ]:
import scipy.io.wavfile
from google.colab import files
import librosa

# output waveform
phase = df_subset.loc[index, "metadata"]["phase"]
#phase = scalers["input_features_phase"].inverse_transform(phase.reshape(1, -1)).reshape(input_temp.shape)
print(np.max(phase))
print(np.min(phase))

# reverse log scale
out_spectrogram = librosa.db_to_amplitude(out_spectrogram)
signal = magphase_to_waveform(out_spectrogram, phase, 44100 * 2)

# Save as WAV file
output_filename = f"denoised_audio_{index}:{snr_db}.wav"
scipy.io.wavfile.write(output_filename, rate=44100, data=signal)  # 16-bit PCM

# Download the file
files.download(output_filename)

tar_track = librosa.db_to_amplitude(tar_track)
signal = magphase_to_waveform(tar_track, phase, 44100 * 2)

# Save as WAV file
output_filename = f"audio_{index}:{snr_db}.wav"
scipy.io.wavfile.write(output_filename, rate=44100, data=signal)  # 16-bit PCM

# Download the file
files.download(output_filename)

in_spectrogram = librosa.db_to_amplitude(in_spectrogram)
signal = magphase_to_waveform(in_spectrogram, phase, 44100 * 2)

# Save as WAV file
output_filename = f"noisy_audio_{index}:{snr_db}.wav"
scipy.io.wavfile.write(output_filename, rate=44100, data=signal)  # 16-bit PCM

# Download the file
files.download(output_filename)

## Plot Results

---